Raj Subramani (19 March 2020) [raj@flumaion.com]

Updated with TFF examples:
Cyril Chimisov (31 March 2020) [cyrilchimisov@google.com]

In [0]:
import math
import QuantLib as ql
from QuantLib import *

# Import TF and TFF
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tf_quant_finance as tff
dates = tff.experimental.dates

##### Below are some extra functions and objects that will be added to TFF 

These are 

  - `DepositCurve` object to fit a yield curve to a set of deposit rates;
  - `get_fixings` helper to cpumpute the fixings at the specified times;
  - `VanillaSwap` object to describe and fit a vanilla swap.

In [0]:
#@title TFF DepositCurve object

class DepositCurve:
  """Discounting curve object that performs fitting to the input deposits."""
  def __init__(self,
               *,
               calc_date,
               deposit_rates,
               settlement_days,
               calendar,
               day_count,
               bussiness_convention,
               periods=None,
               maturity_dates=None,
               dtype=None):
    """Initializer.
    
    Args:
      calc_date: An instance of `DateTensor` of zero shape. The reference date
        to which perform the discounting.
      deposit_rates: A rank 1 `Tensor` of real `dtype`.
      settlement_days: An integer `Tensor` with the shape broadcastable with
        deposit_rates.
      calendar: An instance of `dates.HolidayCalendar2`.
      day_count: A daycount convention. One of `dates.daycounts`.
      bussiness_convention: One of the `dates.BusinessDayConvention` business
        day count conventions.
      periods: An instance of `dates.periods.PeriodTensor`. Either this or
        maturities of the deposit rates should be supplied. Determine the
        maturities of the deposits.
      maturity_dates: An instance of `DateTensor` with the shape broadcastable 
        with `deposit_rates`.
      dtype: A `dtype` for the underlying real `Tensor`s.
        Default value: None which maps to the `dtype` inferred by TensorFlow.
    """  
    if ((maturity_dates is not None and periods is not None)
        or (maturity_dates is None and periods is None)):
      raise ValueError("Either `maturity_dates` or time `periods` shoud be "
                       "supplied but not both.")
    self._calc_date = calc_date
    self._deposit_rates = deposit_rates
    self._periods = periods
    self._settlement_days = settlement_days
    self._calendar = calendar
    self._day_count = day_count
    self._maturity_dates = maturity_dates
    self._dtype = dtype
    self._bussiness_convention = bussiness_convention
  
  def get_value_dates(self):
    """Returns the value dates for deposits adjsuted for settlement dates."""
    return self._calendar.add_business_days(
        self._calc_date, self._settlement_days,
        roll_convention=self._bussiness_convention)

  def get_maturity_dates(self):
    """Returns the maturity dates for deposits adjsuted for settlement dates."""
    if self._periods is None:
      return self._calendar.add_business_days(
          self._maturity_dates, self._settlement_days,
          roll_convention=self._bussiness_convention)
    else:
      return self._calendar.add_period_and_roll(
         self.get_value_dates(), self._periods,
         roll_convention=self._bussiness_convention)
    
  def get_nodes(self):
    """Computes the interpolation nodes for the discounting curve.
    
    Log-linear interpolation is assumed. All `get_value_dates()` should be
    less than `min(get_maturity_dates())`. Also, it is assumed that
    `min(get_maturity_dates()) = get_maturity_dates()[0]`. The `nodes` are
    the time points between which the discounting function is log-linear.

    Returns:
      A 2-tuple of node dates and the corresponding values.
    """
    value_dates = self.get_value_dates()
    maturity_dates = self.get_maturity_dates()
    year_fractions = self._day_count(
        start_date=value_dates, end_date=maturity_dates, dtype=self._dtype) 
    effective_yields = 1 + rates * year_fractions
    # Coefficient `beta` such that
    # log(disc(t)) = beta * t for t <= min(maturity_dates) 
    beta = -tf.math.log(effective_yields[0]) / year_fractions[0]
    initial_discounts = tf.math.exp(
        beta * self._day_count(start_date=calc_date_tf,
                               end_date=value_dates, dtype=self._dtype))
    nodes = initial_discounts / effective_yields
    node_dates = dates.DateTensor.concat([self._calc_date.expand_dims(0),
                                          maturity_dates], axis=0)
    nodes_vals = tf.concat([[1.], nodes], axis=0)
    return node_dates, nodes_vals

  def discount(self, date_tensor=None, t=None):
    """Discounting function that accepts either DateTensor or real Tensor."""
    if date_tensor is None and t is None:
      raise ValueError("Either `date_tensor` or time `t` shoud be supplied.")
    if date_tensor is not None and t is not None: 
      raise ValueError("Either `date_tensor` or time `t` shoud be supplied "
                       "but not both.")
    node_dates, node_vals = self.get_nodes()
    if t is None:
      x = self._day_count(
         start_date=self._calc_date, end_date=date_tensor, dtype=self._dtype)
    else:
      x = tf.convert_to_tensor(t)
    if not x.shape.as_list():
      x = tf.expand_dims(x, axis=-1)
    x_data = self._day_count(
        start_date=self._calc_date, end_date=node_dates, dtype=self._dtype)
    y_data = tf.math.log(node_vals)
    interpolated_values = tff.math.interpolation.linear.interpolate(
        x=x,
        x_data=x_data,
        y_data=y_data)
    return tf.math.exp(interpolated_values)



In [0]:
#@title TFF Fixing funtion and VanillaSwap class

def get_fixings(*, dates_tensor, discount_fn, day_count, dtype):
  """Computes fixings implied by the input dates tensor and a discounting curve.

  Given dates `[d1, d2, .. , dn]` computes forward rates `fwd_rates` between
  `[d_i, d_{i+1}]` for `i=0,..., n-1` and returns the corresponding deposit
  rates defined as `1 + deposit_rates * day_count(d_i, d_{i+1}) = fwd_rates`.

  Args:
    dates_tensor: `DateTensor` of shape `[batch_shape, num_dates]`.
    discount_fn: A callable that maps `DateTensor` to a real number of `dtype`
      which corresponds to discounting.
    day_count: A daycounting function.
    dtype: Output `dtype`.
  
  Returns:
    A `Tensor` of the specified `dtype` and of shape
    `[batch_shape, num_dates - 1]` that correponds to the deposit rates at
    `dates_tensor[1:]`.
  """ 
  start_dates = dates_tensor[..., :-1]
  end_dates = dates_tensor[..., 1:]
  disc_start = discount_fn(start_dates)
  disc_end = discount_fn(end_dates)
  t = day_count(start_date=start_dates, end_date=end_dates, dtype=dtype)
  fixings = tf.where(t > 1e-8,
                     (disc_start/disc_end - 1.0) / t,
                     0.0)
  return fixings

class VanillaSwap:
  """Simple interest rate swap."""
  def __init__(self,
               *,
               calc_date,
               fixed_leg_dates,
               float_leg_dates,
               fixed_leg_rates,
               float_leg_rates,
               notional,
               day_count,
               discount_fn,
               dtype=None):
    """Initializer.
    
    Args:
      calc_date: An instance of `DateTensor` of zero shape. The reference date
        to which perform the discounting.
      fixed_leg_dates: A `DateTensor` of shape `[batch_shape, n]` representing
        the cashflow dates of the fixed leg including the `calc_date` as the
        first entry for each swap in the batch.
      float_leg_dates: A `DateTensor` of shape `[batch_shape, n]` representing
        the cashflow dates of the float leg including the `calc_date` as the
        first entry for each swap in the batch. 
      fixed_leg_rates: A real `Tensor` of shape brodcastable with
        `[batch_shape, n]` representing the fixed rates of the swap.
      float_leg_rates: A real `Tensor` of shape brodcastable with
        `[batch_shape, n]` and of the same dtype as `fixed_leg_rates`.
        Represents the float rates of the swap.
      notional: A real `Tensor` of shape brodcastable with `[batch_shape, n]`
        and of the same dtype as `fixed_leg_rates`. Represents the notional of
        the swap.
      day_count: A daycount convention. One of `dates.daycounts`.
      discount_fn: A callable that maps `DateTensor` to a real number of the
        same `dtype` as `fixed_leg_rates` which corresponds to the discounting
        function.
      dtype: A `dtype` for the underlying real `Tensor`s.
        Default value: None which maps to the `dtype` inferred by TensorFlow.
    """  
    self._calc_date = calc_date
    self._fixed_leg_dates= fixed_leg_dates
    self._float_leg_dates = float_leg_dates
    self._fixed_leg_rates = fixed_leg_rates
    self._float_leg_rates = float_leg_rates
    self._notional = notional
    self._day_count = day_count
    self._discount_fn = discount_fn
    self._dtype = dtype

  def fixed_cashflows(self):
    """Returns all fixed cashflows at `fixed_leg_dates`."""
    start_dates = self._fixed_leg_dates[:-1]
    end_dates = self._fixed_leg_dates[1:]
    t = self._day_count(
        start_date=start_dates, end_date=end_dates, dtype=self._dtype)
    return notional * self._fixed_leg_rates * t

  def float_cashflows(self):
    """Returns all float cashflows at `float_leg_dates`."""
    start_dates = self._float_leg_dates[:-1]
    end_dates = self._float_leg_dates[1:]
    t = self._day_count(
        start_date=start_dates, end_date=end_dates, dtype=self._dtype)
    return notional * self._float_leg_rates * t

  def float_leg_present_value(self):
    """Returns the value of the float leg discounted to `self.calc_date`."""
    payment_dates = self._float_leg_dates[1:]
    cashflows = self.float_cashflows()
    return tf.reduce_sum(cashflows * self._discount_fn(payment_dates), -1)

  def fixed_leg_present_value(self):
    """Returns the value of the fixed leg discounted to `self.calc_date`."""
    payment_dates = self._fixed_leg_dates[1:]
    cashflows = self.fixed_cashflows()
    return tf.reduce_sum(cashflows * self._discount_fn(payment_dates))

  def price(self):
    """Returns the value of the swap discounted to `self.calc_date`."""
    return self.float_leg_present_value() - self.fixed_leg_present_value()


### Construct Yield, Index, Forward and Discount Curves

In [674]:
calc_date = ql.Date(9,9,2015)
ql.Settings.instance().evaluationDate = calc_date
ust_mat=[ql.Period(3, ql.Months),
         ql.Period(6, ql.Months),
         ql.Period(9, ql.Months),
         ql.Period(1, ql.Years),
         ql.Period(2, ql.Years)]
ust_rates=[0.0004, 0.0025, 0.0032, 0.004, 0.0075]
calendar = ql.WeekendsOnly()
bussiness_convention = ql.Unadjusted
day_count = ql.Actual360()
end_of_month = False  # TFF does not support that yet
settlement_days = 2

rate_helpers = [ql.DepositRateHelper(
    ql.QuoteHandle(ql.SimpleQuote(r)),
    m,
    settlement_days,
    calendar,
    bussiness_convention,
    end_of_month,
    day_count
    )
    for r,m in zip(ust_rates, ust_mat)
   ]

discount_curve = ql.PiecewiseLogLinearDiscount(calc_date, rate_helpers, day_count)
forward_curve = ql.PiecewiseFlatForward(calc_date, rate_helpers, day_count)




swap_pricing_libor_curve = ql.RelinkableYieldTermStructureHandle()
swap_pricing_libor_curve.linkTo(discount_curve)
libor_3M_index = ql.USDLibor(ql.Period(3, Months), swap_pricing_libor_curve)

print("\nForward Curve")
for n in forward_curve.nodes():
    print (n)
print("\nDiscount Curve")
for n in discount_curve.nodes():
    print (n)

total_days = calendar.businessDaysBetween(calc_date,ql.Date(7,1,2016))
print("\ntotal days: ", total_days)



Forward Curve
(Date(9,9,2015), 0.00039997977913916704)
(Date(11,12,2015), 0.00039997977913916704)
(Date(11,3,2016), 0.004596863158475467)
(Date(11,6,2016), 0.004576318213765027)
(Date(11,9,2016), 0.0063619267705553335)
(Date(11,9,2017), 0.010904495522554111)

Discount Curve
(Date(9,9,2015), 1.0)
(Date(11,12,2015), 0.9998966772286497)
(Date(11,3,2016), 0.9987354872073664)
(Date(11,6,2016), 0.9975681452543673)
(Date(11,9,2016), 0.995947591022429)
(Date(11,9,2017), 0.9849970929971511)

total days:  86


#### TFF alternative


In [675]:
# Calculation date presented as a DateTensor
calc_date_tf = dates.from_year_month_day(2015, 9, 9)
# Corresponding ql.WeekendsOnly calendar
calendar_tf = dates.HolidayCalendar2(
    weekend_mask=dates.WeekendMask.SATURDAY_SUNDAY)
# Actual360 day count convention
day_count_tf = dates.daycounts.actual_360
# Business day convention
bussiness_convention_tf = dates.BusinessDayConvention.NONE
# Dtype of the discounting
dtype = tf.float64
# Input deposit rates
deposit_rates = tf.convert_to_tensor(
    [0.0004, 0.0025, 0.0032, 0.004, 0.0075], dtype=dtype)
# Corresponding periods for maturities
periods = dates.periods.months([3, 6, 9, 12, 24])
# Settlement dates
settlement_days = 2
# Create discount curve object
discount_curve_tf = DepositCurve(
    calc_date=calc_date_tf,
    deposit_rates=deposit_rates,
    periods=periods,
    settlement_days=settlement_days,
    calendar=calendar_tf,
    day_count=day_count_tf,
    bussiness_convention=bussiness_convention_tf,
    dtype=dtype)
# Compute the interpolation nodes 
node_dates, node_vals = discount_curve_tf.get_nodes()
node_dates_ql, node_vals_ql = list(zip(*discount_curve.nodes()))
print("Node dates (TFF):\n", node_dates, "\n")
print("Node dates (QL):\n", node_dates_ql, "\n")
print("---------------------------")
print("Node values (TFF):\n", node_vals.numpy().tolist(), "\n")
print("Node values (QL):\n", node_vals_ql)

Node dates (TFF):
 DateTensor: shape=(6,), contents=array([[2015,    9,    9],
       [2015,   12,   11],
       [2016,    3,   11],
       [2016,    6,   11],
       [2016,    9,   11],
       [2017,    9,   11]], dtype=int32) 

Node dates (QL):
 (Date(9,9,2015), Date(11,12,2015), Date(11,3,2016), Date(11,6,2016), Date(11,9,2016), Date(11,9,2017)) 

---------------------------
Node values (TFF):
 [1.0, 0.9998966772285539, 0.9987354872073644, 0.9975681452543653, 0.9959475910224269, 0.9849970929971491] 

Node values (QL):
 (1.0, 0.9998966772286497, 0.9987354872073664, 0.9975681452543673, 0.995947591022429, 0.9849970929971511)


In [676]:
# Sanity check (interpolation)
print("TF discount: ", discount_curve_tf.discount(
    calendar_tf.add_business_days(calc_date_tf, [150])).numpy()[0],
      "\nQL discount: ", discount_curve.discount(
          calendar.advance(calc_date, Period(150, Days))))

TF discount:  0.9984054478168319 
QL discount:  0.9984054478168339


In [677]:
# Sanity check (interpolation)
print("TF discount: ", discount_curve_tf.discount(
    calendar_tf.add_business_days(calc_date_tf, [432])).numpy()[0],
      "\nQL discount: ", discount_curve.discount(
          calendar.advance(calc_date, Period(432, Days))))

TF discount:  0.9888534435342962 
QL discount:  0.9888534435342982


### Define Swap (Monocurrency, 3m Libor), Fixed rate = 0.039, Spread=0.0

In [681]:
notional=1000000
fixed_rate = 0.0039
fixed_leg_dcc = day_count
float_spread = 0.0
float_leg_dcc = day_count

settlement_date = calendar.advance(calc_date, ql.Period(settlement_days, Days))
maturity_date = calendar.advance(settlement_date, ql.Period(1, Years))
print(settlement_date)
print(maturity_date)

fixed_leg_tenor = ql.Period(3, ql.Months)
float_leg_tenor = ql.Period(3, ql.Months)

fixed_leg_schedule = ql.Schedule(
    settlement_date, 
    maturity_date, 
    fixed_leg_tenor, 
    calendar, 
    ql.ModifiedFollowing, 
    ql.ModifiedFollowing,
    ql.DateGeneration.Forward, 
    False)

float_leg_schedule = ql.Schedule(
    settlement_date, 
    maturity_date, 
    float_leg_tenor, 
    calendar, 
    ql.ModifiedFollowing, 
    ql.ModifiedFollowing,
    ql.DateGeneration.Forward, 
    False)

ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, 
                         notional, 
                         fixed_leg_schedule, 
                         fixed_rate, 
                         fixed_leg_dcc,
                         float_leg_schedule,
                         libor_3M_index,
                         float_spread,
                         float_leg_dcc
                        )

swap_engine = ql.DiscountingSwapEngine(swap_pricing_libor_curve)
ir_swap.setPricingEngine(swap_engine)

print (ir_swap.NPV())
print(ir_swap.fairRate())
print(calendar.businessDaysBetween(settlement_date,maturity_date))



September 11th, 2015
September 12th, 2016
112.4061367708573
0.004010481275521539
261


In [684]:
# Float rate cashflows
for i, cf in enumerate(ir_swap.leg(1)):
    print ("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

 1    December 11th, 2015      101.11
 2    March 11th, 2016       1162.66
 3    June 13th, 2016        1205.57
 4    September 12th, 2016     1622.09


In [685]:
# Fixed rate cashflows
for i, cf in enumerate(ir_swap.leg(0)):
    print ("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

 1    December 11th, 2015      985.83
 2    March 11th, 2016        985.83
 3    June 13th, 2016        1018.33
 4    September 12th, 2016      985.83


##### TFF alternative

In [0]:

# Create the TFF VanillaSwap
period = dates.periods.years(1)
bussiness_convention_tf = dates.BusinessDayConvention.MODIFIED_FOLLOWING
# Start date of the swap
start_date = calendar_tf.add_business_days(
    calc_date_tf, settlement_days, roll_convention=bussiness_convention_tf)
# Maturity date of the swap
end_date = calendar_tf.add_period_and_roll(
    start_date, period, roll_convention=bussiness_convention_tf)
# Exchange schedules
schedule = dates.PeriodicSchedule(
    start_date=start_date, end_date=end_date,
    tenor=dates.periods.months(3), roll_convention=bussiness_convention_tf)
schedule_dates = calendar_tf.roll_to_business_day(schedule.dates(),
                                                  bussiness_convention_tf)
# Compute the float let rates
float_leg_rates = get_fixings(dates_tensor=dates_tensor,
        day_count=day_count_tf,
        discount_fn=discount_curve_tf.discount,
        dtype=dtype)
# Create a swap object (can price a batch of swaps)
swap_tf = VanillaSwap(
    calc_date=calc_date_tf,
    fixed_leg_dates=schedule_dates,
    float_leg_dates=schedule_dates,
    fixed_leg_rates=0.0039,
    float_leg_rates=float_leg_rates,
    notional=1000000,
    day_count=day_count_tf,
    discount_fn=discount_curve_tf.discount,
    dtype=dtype)

In [697]:
# Float rate cashflows
print("                              QuantLib     TFF")
for i, (ql_leg, tff_leg) in enumerate(zip(ir_swap.leg(1),
                                          swap_tf.float_cashflows().numpy())):
    print ("%2d    %-18s  %10.2f  %10.2f"%(i+1, ql_leg.date(),
                                           ql_leg.amount(), tff_leg))

                              QuantLib     TFF
 1    December 11th, 2015      101.11      101.11
 2    March 11th, 2016       1162.66     1162.66
 3    June 13th, 2016        1205.57     1205.57
 4    September 12th, 2016     1622.09     1622.09


In [698]:
# Fixed rate cashflows
print("                              QuantLib     TFF")
for i, (ql_leg, tff_leg) in enumerate(zip(ir_swap.leg(0),
                                          swap_tf.fixed_cashflows().numpy())):
    print ("%2d    %-18s  %10.2f  %10.2f"%(i+1, ql_leg.date(),
                                           ql_leg.amount(), tff_leg))

                              QuantLib     TFF
 1    December 11th, 2015      985.83      985.83
 2    March 11th, 2016        985.83      985.83
 3    June 13th, 2016        1018.33     1018.33
 4    September 12th, 2016      985.83      985.83


In [699]:
# Swap price comparison
print("TFF float leg price: ", swap_tf.float_leg_present_value().numpy())
print("QL float leg price: ", ir_swap.floatingLegNPV())
print("-----------------------------")
print("TFF fixed leg price: ", swap_tf.fixed_leg_present_value().numpy())
print("QL fixed leg price: ", ir_swap.fixedLegNPV())
print("-----------------------------")
print("TFF swap price: ", swap_tf.price().numpy())
print("QL swap price: ", ir_swap.NPV())
print("Relative error: ",
      abs(swap_tf.price().numpy() - ir_swap.NPV()) / ir_swap.NPV())

TFF float leg price:  4080.353930067805
QL float leg price:  4080.353930067715
-----------------------------
TFF fixed leg price:  3967.9477932970226
QL fixed leg price:  -3967.9477932968575
-----------------------------
TFF swap price:  112.40613677078227
QL swap price:  112.4061367708573
Relative error:  6.675197195792241e-13


### Valuation date 9th September 2015

In [0]:
import pandas as pd
col01='Days in payment leg'
col02='Payment Fraction'
col03='Forward rate'
col04='Discount rate'
col05='Calculated Fix C/flow'
col06='QL Fix C/flow'
col07='Fix diff'
col08='Calculated Float C/flow'
col09='QL Float C/flow'
col10='Float diff'
df = pd.DataFrame(columns=[col01, col02, col03, col04, col05, col06, col07, col08, col09, col10])

previous_payment_date = settlement_date
total_days = calendar.businessDaysBetween(settlement_date, maturity_date)
for fix, flt in zip(ir_swap.leg(0),ir_swap.leg(1)):
    elapsed_days = int(calendar.businessDaysBetween(previous_payment_date, flt.date()))
    forward_rate = forward_curve.forwardRate(previous_payment_date, flt.date(), day_count, ql.Compounded).rate()
    payment_fraction = elapsed_days/total_days
    equiv_fwd_rate = forward_rate * payment_fraction
    discount_rate = forward_curve.discount(flt.date())
    calculated_fixed_cash_flow = notional * fixed_rate * payment_fraction * discount_rate
    calculated_float_cash_flow = notional * equiv_fwd_rate * discount_rate
    df.loc[fix.date()] = [
        elapsed_days, 
        payment_fraction, 
        forward_rate, 
        discount_rate,
        calculated_fixed_cash_flow,
        fix.amount(),
        (fix.amount() - calculated_fixed_cash_flow),
        calculated_float_cash_flow,
        flt.amount(),
        (flt.amount() - calculated_float_cash_flow),
   ]
    
    previous_payment_date = flt.date()
df
    
    

,Days in payment leg,Payment Fraction,Forward rate,Discount rate,Calculated Fix C/flow,QL Fix C/flow,Fix diff,Calculated Float C/flow,QL Float C/flow,Float diff
"December 11th, 2015",62.0,0.247012,0.000400,0.999897,963.247078,985.833333,22.586255,98.809337,101.106056,2.296719
"March 11th, 2016",61.0,0.243028,0.004607,0.998735,946.610249,985.833333,39.223085,1118.321693,1162.660220,44.338527
"June 13th, 2016",65.0,0.258964,0.004625,0.997533,1007.468474,1018.333333,10.864859,1194.747101,1205.573702,10.826601
"September 12th, 2016",63.0,0.250996,0.006432,0.995917,974.888092,985.833333,10.945241,1607.926673,1622.086119,14.159446


### Valuation date 7th January 2016 (add fixing)

In [0]:
df = df.iloc[0:0]

fixing_date = calendar.advance(settlement_date, Period(-1*settlement_days, Days))
print(fixing_date)
advance_by = calendar.businessDaysBetween(settlement_date, Date(7,1,2016))
calc_date = calendar.advance(settlement_date, Period(advance_by,Days))
print(calc_date)
while (fixing_date < calc_date):
    if (libor_3M_index.isValidFixingDate(fixing_date)):
        fixing_rate = forward_curve.forwardRate(fixing_date, TARGET().advance(fixing_date,1,Days), Actual360(), Simple).rate()
        libor_3M_index.addFixing(fixing_date, fixing_rate, True)
        #print(fixing_date, fixing_rate)
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
    else:
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
        



September 9th, 2015
January 7th, 2016


In [0]:
Settings.instance().evaluationDate = calc_date

In [0]:
discount_curve = ql.PiecewiseLogLinearDiscount(calc_date, rate_helpers, day_count)
swap_pricing_libor_curve.linkTo(discount_curve)

print (ir_swap.NPV())
print(ir_swap.fairRate())

col01='Days in payment leg'
col02='Payment Fraction'
col03='Forward rate'
col04='Discount rate'
col05='Calculated Fix C/flow'
col06='QL Fix C/flow'
col07='Fix diff'
col08='Calculated Float C/flow'
col09='QL Float C/flow'
col10='Float diff'
df = pd.DataFrame(columns=[col01, col02, col03, col04, col05, col06, col07, col08, col09, col10])

previous_payment_date = settlement_date
for fix, flt in zip(ir_swap.leg(0),ir_swap.leg(1)):
    print(fix.date())
    elapsed_days = int(calendar.businessDaysBetween(previous_payment_date, flt.date()))
    forward_rate = forward_curve.forwardRate(previous_payment_date, flt.date(), day_count, ql.Compounded).rate()
    payment_fraction = elapsed_days/total_days
    equiv_fwd_rate = forward_rate * payment_fraction
    discount_rate = forward_curve.discount(flt.date())
    calculated_fixed_cash_flow = notional * fixed_rate * payment_fraction * discount_rate
    calculated_float_cash_flow = notional * equiv_fwd_rate * discount_rate
    previous_payment_date = flt.date()
    df.loc[fix.date()] = [
        elapsed_days, 
        payment_fraction, 
        forward_rate, 
        discount_rate,
        calculated_fixed_cash_flow,
        fix.amount(),
        (fix.amount() - calculated_fixed_cash_flow),
        calculated_float_cash_flow,
        flt.amount(),
        (flt.amount() - calculated_float_cash_flow),
   ]
df

-890.7196646672974
0.002737011895371293
December 11th, 2015
March 11th, 2016
June 13th, 2016
September 12th, 2016


,Days in payment leg,Payment Fraction,Forward rate,Discount rate,Calculated Fix C/flow,QL Fix C/flow,Fix diff,Calculated Float C/flow,QL Float C/flow,Float diff
"December 11th, 2015",62.0,0.247012,0.000400,0.999897,963.247078,985.833333,22.586255,98.809337,101.106056,2.296719
"March 11th, 2016",61.0,0.243028,0.000400,0.999796,947.615020,985.833333,38.218313,97.205810,101.106056,3.900246
"June 13th, 2016",65.0,0.258964,0.003218,0.998957,1008.906989,1018.333333,9.426344,832.464969,839.245726,6.780757
"September 12th, 2016",63.0,0.250996,0.004593,0.997801,976.731606,985.833333,9.101727,1150.328390,1159.060681,8.732291


import pandas as pd
col01='Days in payment leg'
col02='Payment Fraction'
col03='Forward rate'
col04='Discount rate'
col05='Calculated Fix C/flow'
col06='QL Fix C/flow'
col07='Fix diff'
col08='Calculated Float C/flow'
col09='QL Float C/flow'
col10='Float diff'
df = pd.DataFrame(columns=[col01, col02, col03, col04, col05, col06, col07, col08, col09, col10])

previous_payment_date = settlement_date
total_days = calendar.businessDaysBetween(settlement_date, maturity_date)
for fix, flt in zip(ir_swap.leg(0),ir_swap.leg(1)):
    elapsed_days = int(calendar.businessDaysBetween(previous_payment_date, flt.date()))
    forward_rate = forward_curve.forwardRate(previous_payment_date, flt.date(), day_count, ql.Compounded).rate()
    payment_fraction = elapsed_days/total_days
    equiv_fwd_rate = forward_rate * payment_fraction
    discount_rate = forward_curve.discount(flt.date())
    calculated_fixed_cash_flow = notional * fixed_rate * payment_fraction * discount_rate
    calculated_float_cash_flow = notional * equiv_fwd_rate * discount_rate
    df.loc[fix.date()] = [
        elapsed_days, 
        payment_fraction, 
        forward_rate, 
        discount_rate,
        calculated_fixed_cash_flow,
        fix.amount(),
        (fix.amount() - calculated_fixed_cash_flow),
        calculated_float_cash_flow,
        flt.amount(),
        (flt.amount() - calculated_float_cash_flow),
   ]
    
    previous_payment_date = flt.date()
df
    
    

### Valuation date 23rd April 2016 (add fixing)

In [0]:
df = df.iloc[0:0]

fixing_date = calendar.advance(settlement_date, Period(-1*settlement_days, Days))
print(fixing_date)
advance_by = calendar.businessDaysBetween(settlement_date, Date(23,4,2016))
calc_date = calendar.advance(settlement_date, Period(advance_by,Days))
print(calc_date)
while (fixing_date < calc_date):
    if (libor_3M_index.isValidFixingDate(fixing_date)):
        fixing_rate = forward_curve.forwardRate(fixing_date, TARGET().advance(fixing_date,1,Days), Actual360(), Simple).rate()
        libor_3M_index.addFixing(fixing_date, fixing_rate, True)
        #print(fixing_date, fixing_rate)
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
    else:
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
        


September 9th, 2015
April 25th, 2016


In [0]:
Settings.instance().evaluationDate = calc_date

In [0]:
discount_curve = ql.PiecewiseLogLinearDiscount(calc_date, rate_helpers, day_count)
swap_pricing_libor_curve.linkTo(discount_curve)

print (ir_swap.NPV())
print(ir_swap.fairRate())

col01='Days in payment leg'
col02='Payment Fraction'
col03='Forward rate'
col04='Discount rate'
col05='Calculated Fix C/flow'
col06='QL Fix C/flow'
col07='Fix diff'
col08='Calculated Float C/flow'
col09='QL Float C/flow'
col10='Float diff'
df = pd.DataFrame(columns=[col01, col02, col03, col04, col05, col06, col07, col08, col09, col10])

previous_payment_date = settlement_date
for fix, flt in zip(ir_swap.leg(0),ir_swap.leg(1)):
    print(fix.date())
    elapsed_days = int(calendar.businessDaysBetween(previous_payment_date, flt.date()))
    forward_rate = forward_curve.forwardRate(previous_payment_date, flt.date(), day_count, ql.Compounded).rate()
    payment_fraction = elapsed_days/total_days
    equiv_fwd_rate = forward_rate * payment_fraction
    discount_rate = forward_curve.discount(flt.date())
    calculated_fixed_cash_flow = notional * fixed_rate * payment_fraction * discount_rate
    calculated_float_cash_flow = notional * equiv_fwd_rate * discount_rate
    previous_payment_date = flt.date()
    df.loc[fix.date()] = [
        elapsed_days, 
        payment_fraction, 
        forward_rate, 
        discount_rate,
        calculated_fixed_cash_flow,
        fix.amount(),
        (fix.amount() - calculated_fixed_cash_flow),
        calculated_float_cash_flow,
        flt.amount(),
        (flt.amount() - calculated_float_cash_flow),
   ]
df

-1253.1788403366695
0.001460473948487446
December 11th, 2015
March 11th, 2016
June 13th, 2016
September 12th, 2016


,Days in payment leg,Payment Fraction,Forward rate,Discount rate,Calculated Fix C/flow,QL Fix C/flow,Fix diff,Calculated Float C/flow,QL Float C/flow,Float diff
"December 11th, 2015",62.0,0.247012,0.000400,0.999897,963.247078,985.833333,22.586255,98.809337,101.106056,2.296719
"March 11th, 2016",61.0,0.243028,0.000400,0.999796,947.615020,985.833333,38.218313,97.205810,101.106056,3.900246
"June 13th, 2016",65.0,0.258964,0.000400,0.999691,1009.648258,1018.333333,8.685076,103.569144,104.439223,0.870079
"September 12th, 2016",63.0,0.250996,0.002559,0.999046,977.950146,985.833333,7.883187,641.705881,646.261066,4.555185


### Valuation date 22nf June 2016 (add fixing)

In [0]:
df = df.iloc[0:0]

fixing_date = calendar.advance(settlement_date, Period(-1*settlement_days, Days))
print(fixing_date)
advance_by = calendar.businessDaysBetween(settlement_date, Date(22,6,2016))
calc_date = calendar.advance(settlement_date, Period(advance_by,Days))
print(calc_date)
while (fixing_date < calc_date):
    if (libor_3M_index.isValidFixingDate(fixing_date)):
        fixing_rate = forward_curve.forwardRate(fixing_date, TARGET().advance(fixing_date,1,Days), Actual360(), Simple).rate()
        libor_3M_index.addFixing(fixing_date, fixing_rate, True)
        #print(fixing_date, fixing_rate)
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
    else:
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
        


September 9th, 2015
June 22nd, 2016


In [0]:
Settings.instance().evaluationDate = calc_date

In [0]:
discount_curve = ql.PiecewiseLogLinearDiscount(calc_date, rate_helpers, day_count)
swap_pricing_libor_curve.linkTo(discount_curve)

print (ir_swap.NPV())
print(ir_swap.fairRate())

col01='Days in payment leg'
col02='Payment Fraction'
col03='Forward rate'
col04='Discount rate'
col05='Calculated Fix C/flow'
col06='QL Fix C/flow'
col07='Fix diff'
col08='Calculated Float C/flow'
col09='QL Float C/flow'
col10='Float diff'
df = pd.DataFrame(columns=[col01, col02, col03, col04, col05, col06, col07, col08, col09, col10])

previous_payment_date = settlement_date
for fix, flt in zip(ir_swap.leg(0),ir_swap.leg(1)):
    print(fix.date())
    elapsed_days = int(calendar.businessDaysBetween(previous_payment_date, flt.date()))
    forward_rate = forward_curve.forwardRate(previous_payment_date, flt.date(), day_count, ql.Compounded).rate()
    payment_fraction = elapsed_days/total_days
    equiv_fwd_rate = forward_rate * payment_fraction
    discount_rate = forward_curve.discount(flt.date())
    calculated_fixed_cash_flow = notional * fixed_rate * payment_fraction * discount_rate
    calculated_float_cash_flow = notional * equiv_fwd_rate * discount_rate
    previous_payment_date = flt.date()
    df.loc[fix.date()] = [
        elapsed_days, 
        payment_fraction, 
        forward_rate, 
        discount_rate,
        calculated_fixed_cash_flow,
        fix.amount(),
        (fix.amount() - calculated_fixed_cash_flow),
        calculated_float_cash_flow,
        flt.amount(),
        (flt.amount() - calculated_float_cash_flow),
   ]
df

-884.6466767401291
0.0003999800013602421
December 11th, 2015
March 11th, 2016
June 13th, 2016
September 12th, 2016


,Days in payment leg,Payment Fraction,Forward rate,Discount rate,Calculated Fix C/flow,QL Fix C/flow,Fix diff,Calculated Float C/flow,QL Float C/flow,Float diff
"December 11th, 2015",62.0,0.247012,0.0004,0.999897,963.247078,985.833333,22.586255,98.809282,101.106056,2.296773
"March 11th, 2016",61.0,0.243028,0.0004,0.999796,947.615020,985.833333,38.218313,97.205756,101.106056,3.900300
"June 13th, 2016",65.0,0.258964,0.0004,0.999691,1009.648258,1018.333333,8.685075,103.569086,104.439223,0.870136
"September 12th, 2016",63.0,0.250996,0.0004,0.999590,978.483222,985.833333,7.350111,100.372197,101.106056,0.733859


### Valuation date 5th September 2016 (add fixing)

In [0]:
df = df.iloc[0:0]

fixing_date = calendar.advance(settlement_date, Period(-1*settlement_days, Days))
print(fixing_date)
advance_by = calendar.businessDaysBetween(settlement_date, Date(5,9,2016))
calc_date = calendar.advance(settlement_date, Period(advance_by,Days))
print(calc_date)
while (fixing_date < calc_date):
    if (libor_3M_index.isValidFixingDate(fixing_date)):
        fixing_rate = forward_curve.forwardRate(fixing_date, TARGET().advance(fixing_date,1,Days), Actual360(), Simple).rate()
        libor_3M_index.addFixing(fixing_date, fixing_rate, True)
        #print(fixing_date, fixing_rate)
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
    else:
        fixing_date = calendar.advance(fixing_date, Period(1,Days))
        


September 9th, 2015
September 6th, 2016


In [0]:
Settings.instance().evaluationDate = calc_date

In [0]:
discount_curve = ql.PiecewiseLogLinearDiscount(calc_date, rate_helpers, day_count)
swap_pricing_libor_curve.linkTo(discount_curve)

print (ir_swap.NPV())
print(ir_swap.fairRate())

col01='Days in payment leg'
col02='Payment Fraction'
col03='Forward rate'
col04='Discount rate'
col05='Calculated Fix C/flow'
col06='QL Fix C/flow'
col07='Fix diff'
col08='Calculated Float C/flow'
col09='QL Float C/flow'
col10='Float diff'
df = pd.DataFrame(columns=[col01, col02, col03, col04, col05, col06, col07, col08, col09, col10])

previous_payment_date = settlement_date
for fix, flt in zip(ir_swap.leg(0),ir_swap.leg(1)):
    print(fix.date())
    elapsed_days = int(calendar.businessDaysBetween(previous_payment_date, flt.date()))
    forward_rate = forward_curve.forwardRate(previous_payment_date, flt.date(), day_count, ql.Compounded).rate()
    payment_fraction = elapsed_days/total_days
    equiv_fwd_rate = forward_rate * payment_fraction
    discount_rate = forward_curve.discount(flt.date())
    calculated_fixed_cash_flow = notional * fixed_rate * payment_fraction * discount_rate
    calculated_float_cash_flow = notional * equiv_fwd_rate * discount_rate
    previous_payment_date = flt.date()
    df.loc[fix.date()] = [
        elapsed_days, 
        payment_fraction, 
        forward_rate, 
        discount_rate,
        calculated_fixed_cash_flow,
        fix.amount(),
        (fix.amount() - calculated_fixed_cash_flow),
        calculated_float_cash_flow,
        flt.amount(),
        (flt.amount() - calculated_float_cash_flow),
   ]
df

-884.7214357423887
0.00039997977913800147
December 11th, 2015
March 11th, 2016
June 13th, 2016
September 12th, 2016


,Days in payment leg,Payment Fraction,Forward rate,Discount rate,Calculated Fix C/flow,QL Fix C/flow,Fix diff,Calculated Float C/flow,QL Float C/flow,Float diff
"December 11th, 2015",62.0,0.247012,0.0004,0.999897,963.247078,985.833333,22.586255,98.809337,101.106000,2.296662
"March 11th, 2016",61.0,0.243028,0.0004,0.999796,947.615020,985.833333,38.218313,97.205810,101.106000,3.900190
"June 13th, 2016",65.0,0.258964,0.0004,0.999691,1009.648258,1018.333333,8.685076,103.569144,104.439165,0.870021
"September 12th, 2016",63.0,0.250996,0.0004,0.999590,978.483222,985.833333,7.350111,100.372252,101.106000,0.733747
